In [ ]:
import requests
import json
import pandas as pd
import geopandas as gpd

requestString = 'https://data.lacounty.gov/resource/42ne-gwcj.json?City=LOS ANGELES CA&$limit=1000000'
r = requests.get(requestString)
d = json.loads(r.text)
df_2020 = pd.DataFrame(d)
df_2020

In [ ]:
#keep the desired columns
columns = ['ain','situszip','center_lat','center_lon','rollyear','roll_landvalue','sqftmain','roll_impbaseyear','propertylocation','usecodedescchar1','effectiveyearbuilt','parcelboundarydescription','roll_landbaseyear','location_1']
df_2020_n = pd.DataFrame(df_2020, columns=columns)
df_2020_n

In [ ]:
#convert to a geodataframe
gdf_2020 = gpd.GeoDataFrame(
    df_2020_n, geometry=gpd.points_from_xy(df_2020_n.center_lon, df_2020_n.center_lat, 
                                          crs='EPSG:4326'))
gdf_2020.head()

In [ ]:
#import neighborhood boundary data
nb= gpd.read_file('/Users/jinglanlin/Desktop/School/01 UCLA/04 Spring 2022/UP 229 DS/Github/UDS-Project--Full-Team-/Data/LA_Times_Neighborhood_Boundaries (1)-Copy1.geojson')
nb.head()

In [ ]:
#join neighborhood boundary with land value data
gdf_2020_sjoin = gpd.sjoin(gdf_2020,nb.to_crs('EPSG:4326'),how="left", predicate='intersects')
gdf_2020_sjoin.head()

In [ ]:
#keep data with the following neighborhood: Koreatown, Westlake, Silver Lake, Echo Park, Downtown, Pico Union
Westlake = gdf_2020_sjoin.loc[gdf_2020_sjoin['name'] == 'Westlake']
Westlake = Westlake.drop(['index_right'], axis=1)
Westlake

In [ ]:
Koreatown = gdf_2020_sjoin.loc[gdf_2020_sjoin['name'] == 'Koreatown']
Koreatown = Koreatown.drop(['index_right'], axis=1)
Koreatown

In [ ]:
Pico_Union = gdf_2020_sjoin.loc[gdf_2020_sjoin['name'] == 'Pico-Union']
Pico_Union = Pico_Union.drop(['index_right'], axis=1)
Pico_Union

In [ ]:
Downtown = gdf_2020_sjoin.loc[gdf_2020_sjoin['name'] == 'Downtown']
Downtown = Downtown.drop(['index_right'], axis=1)
Downtown

In [ ]:
Silver_Lake = gdf_2020_sjoin.loc[gdf_2020_sjoin['name'] == 'Silver Lake']
Silver_Lake = Silver_Lake.drop(['index_right'], axis=1)
Silver_Lake

In [ ]:
Echo_Park = gdf_2020_sjoin.loc[gdf_2020_sjoin['name'] == 'Echo Park']
Echo_Park = Echo_Park.drop(['index_right'], axis=1)
Echo_Park

In [ ]:
join1 = pd.concat([Westlake, Koreatown], axis=0).reset_index(drop=True)
join1

In [ ]:
join2 = pd.concat([join1,Pico_Union], axis=0).reset_index(drop=True)
join2

In [ ]:
join3 = pd.concat([join2,Downtown], axis=0).reset_index(drop=True)
join3

In [ ]:
join4 = pd.concat([join3,Silver_Lake], axis=0).reset_index(drop=True)
join4

In [ ]:
#final join of all the neighborhoods to the land value data
finaljoin = pd.concat([join4,Echo_Park], axis=0).reset_index(drop=True)
finaljoin

In [ ]:
#import census tract dataset
tract = gpd.read_file('/Users/jinglanlin/Desktop/School/01 UCLA/04 Spring 2022/UP 229 DS/Github/UDS-Project--Full-Team-/Data/Census_Tracts_2020.geojson')
tract.head()

In [ ]:
#join tracts to land value data with neighborhood
gdf_2020_sjoin = gpd.sjoin(finaljoin,tract.to_crs('EPSG:4326'),how="left", predicate='intersects')
gdf_2020_sjoin =gdf_2020_sjoin.rename(columns={'name': 'neighborhood'})
gdf_2020_sjoin

In [ ]:
#save the file to the folder
gdf_2020_sjoin.to_file('landvalue_2020_new.geojson', driver='GeoJSON') 